In [5]:
import dask.dataframe as dd


In [6]:
ddf = dd.read_csv('/data2st1/junyi/snmc_bedgraph/merged/merge_peak_unified.bedGraph',sep='\t',header=0, assume_missing=True)

In [7]:
ddf.columns

Index(['chrom', 'start', 'end', 'ABC_NN.CGN', 'ACB-BST-FS_D1_Gaba.CGN',
       'ADP-MPO_Trp73_Glut.CGN', 'AD_Serpinb7_Glut.CGN',
       'AHN_Onecut3_Gaba.CGN', 'AHN-RCH-LHA_Otp_Fezf1_Glut.CGN',
       'AHN-SBPV-PVHd_Pdrm12_Gaba.CGN',
       ...
       'TH_Prkcd_Grin2c_Glut.CGN', 'TMv-PMv_Tbx3_Hist-Gaba.CGN',
       'TRS-BAC_Sln_Glut.CGN', 'TU-ARH_Otp_Six6_Gaba.CGN',
       'VCO_Mafa_Meis2_Glut.CGN', 'Vip_Gaba.CGN', 'VLMC_NN.CGN',
       'VMH_Fezf1_Glut.CGN', 'VMH_Nr5a1_Glut.CGN', 'ZI_Pax6_Gaba.CGN'],
      dtype='object', length=277)

In [8]:
matrix = ddf.iloc[:,3:]

In [9]:
matrix = matrix.compute()

In [10]:
# import scipy.sparse as sp
# sparse_matrix = sp.csr_matrix(matrix.values)
sparse_matrix = matrix

In [13]:
import scanpy as sc
adata = sc.AnnData(sparse_matrix)

/home/junyichen/anaconda3/envs/snapatac2/lib/python3.10/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/home/junyichen/anaconda3/envs/snapatac2/lib/python3.10/site-packages/anndata/_core/anndata.py:1754: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [15]:
adata = adata.T

/home/junyichen/anaconda3/envs/snapatac2/lib/python3.10/site-packages/anndata/_core/anndata.py:1756: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [16]:
region_df = ddf.iloc[:,0:3]

In [17]:
region_df = region_df.compute()

In [18]:
region_df['start'] = region_df['start'].astype(int)
region_df['end'] = region_df['end'].astype(int)

In [19]:
region_df

,chrom,start,end
0,chr1,3000826,3000828
1,chr1,3001006,3001019
2,chr1,3001276,3001278
3,chr1,3001628,3001630
4,chr1,3003225,3003380
...,...,...,...
87082,chrY,90840847,90841007
87083,chrY,90841179,90842358
87084,chrY,90842522,90842770
87085,chrY,90842936,90843476


In [20]:
adata.obs_names = ddf.columns[3:]

In [21]:
adata.var = region_df

/home/junyichen/anaconda3/envs/snapatac2/lib/python3.10/site-packages/anndata/_core/anndata.py:747: UserWarning: 
AnnData expects .var.index to contain strings, but got values like:
    [0, 1, 2, 3, 4]

    Inferred to be: integer

  value_idx = self._prep_dim_index(value.index, attr)


In [22]:
adata.var.index = adata.var['chrom'] + ':' + adata.var['start'].astype(str) + '-' + adata.var['end'].astype(str)

In [23]:
adata.var.index = adata.var.index.astype(str)
adata.obs.index = adata.obs.index.astype(str)

In [26]:
adata.var.to_csv('/data2st1/junyi/snmc_bedgraph/merged/region_bed.bed', sep='\t', header=False, index=False)

In [24]:
adata.write_h5ad('/data2st1/junyi/snmc_bedgraph/merged/meth_sum_regions.h5ad')

In [27]:
mapping = {
    'Oligo_NN': 'OPC-Oligo',
    'Microglia_NN': 'Immune',
    'Endo_NN': 'Vascular',
    'Astro-TE_NN': 'Astro-Epen',
    'Astro-OLF_NN': 'Astro-Epen',
    'Astro-NT_NN': 'Astro-Epen',
    'Astroependymal_NN': 'Astro-Epen',
    'Hypendymal_NN': 'Astro-Epen',
    'Ependymal_NN': 'Astro-Epen',
    'Tanycyte_NN': 'Astro-Epen',
    'Bergmann_NN': 'Astro-Epen',
    'CHOR_NN': 'Astro-Epen',
    'Astro-CB_NN': 'Astro-Epen',
    'OPC_NN': 'OPC-Oligo',
    'OEC_NN': 'Astro-Epen',
    'BAM_NN': 'Immune',
    'DC_NN': 'Immune',
    'Lymphoid_NN': 'Immune',
    'VLMC_NN': 'Vascular',
    'ABC_NN': 'Immune',
    'SMC_NN': 'Vascular',
    'Peri_NN': 'Vascular'
}

In [28]:
adata.obs['Subtype'] = adata.obs.index.str.replace('.CGN','')

In [29]:
adata.obs['Celltype.l1'] = adata.obs['Subtype'].map(mapping).fillna('Neuron')


In [30]:
adata.obs['Celltype.l2'] = adata.obs['Subtype'].str.split('_').str[0]+"_"+adata.obs['Subtype'].str.split('_').str[-1]

In [34]:
adata.X.mean()

4.870273599875502

In [38]:
import pybedtools
from pyfaidx import Fasta

# Paths to your BED file and FASTA genome sequence file
bed_file = '/data2st1/junyi/snmc_bedgraph/merged/region_bed.bed'  # Replace with your BED file
fasta_file = '/data2st1/junyi/ref/GRCm38.p6.genome.fa'  # Replace with your FASTA file
fasta_index = '/data2st1/junyi/ref/GRCm38.p6.genome.fa.fai' #Index file of the fasta file

try:
    genome = Fasta(fasta_file, sequence_always_upper=True, rebuild=False, build_index=False) # sequence_always_upper=True to ensure all sequences are uppercase
except Exception as e:
    print("Fasta index file not found. Creating index file")
    genome = Fasta(fasta_file, sequence_always_upper=True, rebuild=True)

def count_c_bases(sequence):
    """Counts the number of 'C' bases in a sequence."""
    return sequence.upper().count('C')  # Ensure case-insensitivity


Fasta index file not found. Creating index file


In [98]:
bed = pybedtools.BedTool(bed_file)
counts = []

# Iterate through each region in the BED file
for interval in bed:
    chrom = interval.chrom
    start = interval.start
    end = interval.end

    # Extract the genomic sequence for the region
    try:
        sequence = genome[chrom][start-1:end].seq
    except KeyError as e:
        print(f"Warning: Chromosome {chrom} not found in FASTA file. Skipping.")
        continue
    except IndexError as e:
        print(f"Warning: Indexing error for {chrom}:{start}-{end}. Skipping.")
        continue

    # Count the number of 'C' bases in the sequence
    c_count = count_c_bases(sequence)

    counts.append(c_count)
    # Print the results (you can modify this to save to a file)


In [100]:
adata.var['C_count'] = counts

In [45]:
adata.write_h5ad('/data2st1/junyi/snmc_bedgraph/merged/meth_sum_regions.h5ad')

In [ ]:
#adata.layers['raw'] = adata.X.copy() 

In [102]:
adata.X = adata.layers['raw'] / adata.var['C_count'].values

In [111]:
adata.layers['raw'].max(axis=1)

array([1280.223943 , 1190.129391 ,  938.749034 , 1102.214862 ,
       1205.490717 , 1311.250237 , 1237.701641 , 1128.234862 ,
       1230.392805 ,  898.253682 , 1154.166254 , 1484.215099 ,
       1484.698233 , 1293.283435 ,  929.948259 ,  868.68483  ,
        865.949477 , 1215.059059 ,  780.631231 ,  929.526633 ,
       1050.766867 , 1458.792721 , 1376.572657 , 1446.557257 ,
       1409.071374 , 1299.763842 ,  998.9035301, 1348.047269 ,
        911.682029 , 1359.946256 , 1478.12818  , 1304.137432 ,
       1167.152821 ,  640.788821 ,  884.770037 , 1139.499388 ,
        776.18189  , 1479.979319 , 1326.635972 , 1364.67393  ,
        983.158961 , 1143.58617  , 1076.575678 , 1311.925773 ,
        889.563252 , 1225.385729 , 1095.435068 , 1069.220209 ,
       1315.532709 , 1459.953368 , 1359.389872 , 1342.625966 ,
       1066.265151 , 1236.817373 ,  853.883559 ,  944.265397 ,
        698.630373 ,  754.49561  , 1241.020984 , 1171.15038  ,
       1206.429986 , 1055.339366 ,  813.770763 ,  862.4

In [114]:
len(adata.X.max(axis=0))

5380485

In [121]:
x_argmax = adata.X.argmax(axis=1)

In [128]:
maxlarge1 = adata.X.max(axis=0) > 1

In [132]:
len(maxlarge1)

5380485

In [135]:
adata.var[maxlarge1].sort_values('C_count', ascending=False)

,chrom,start,end,C_count
chr5:125166307-125166806,chr5,125166307,125166806,97
chr1:185458101-185458724,chr1,185458101,185458724,93
chr13:77109385-77109897,chr13,77109385,77109897,75
chr12:50807500-50808099,chr12,50807500,50808099,63
chrX:107589505-107590059,chrX,107589505,107590059,62
...,...,...,...,...
chr6:58863681-58863683,chr6,58863681,58863683,1
chr6:58863430-58863432,chr6,58863430,58863432,1
chr6:58858146-58858148,chr6,58858146,58858148,1
chr6:58855670-58855672,chr6,58855670,58855672,1


In [96]:
genome[0][3001627:3001630]

>chr1:3001628-3001630
CCG

In [57]:
adata.var['C_count']

chr1:3000826-3000828        1
chr1:3001006-3001019        3
chr1:3001276-3001278        1
chr1:3001628-3001630        1
chr1:3003225-3003380       40
                         ... 
chrY:90840847-90841007     33
chrY:90841179-90842358    276
chrY:90842522-90842770     55
chrY:90842936-90843476    105
chrY:90843638-90844677    237
Name: C_count, Length: 5380485, dtype: int64

In [77]:
adata.layers['raw'].max(axis=0)

array([ 1.983398,  3.978028,  1.988281, ...,  3.      ,  0.      ,
       13.333008])